In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nolds
import itertools

In [4]:
data_mackey  = pd.read_csv("./TimeseriesData/mackey_1000.csv")
data_mackey = data_mackey.head(1000)
data_mackey = np.array(data_mackey, dtype=np.float32)
data_mackey = data_mackey.reshape(-1)

In [5]:
data_lorenz = pd.read_csv("./TimeseriesData/lorenz_1000.csv")
data_lorenz = np.array(data_lorenz, dtype=np.float32)

In [6]:
data_henon = pd.read_csv("./TimeseriesData/henon_1000.csv")
data_henon = data_henon.head(1000)
data_henon = np.array(data_henon, dtype=np.float32)

In [7]:
def determine_mean_frequency(data, plot=False):
    """
    Plots the frequency spectrum of the given dataset and computes the mean frequency.

    Parameters:
    data (numpy.ndarray): The time-series data.
    sampling_rate (float): The sampling rate of the data in Hz.
    """
    # Compute the number of data points
    n = len(data)
    
    # Perform the Fast Fourier Transform (FFT)
    fft_result = np.fft.fft(data)
    
    # Compute the frequency values
    frequencies = np.fft.fftfreq(n, d=1)
    
    # Compute the magnitude of the FFT (only positive frequencies)
    magnitude = np.abs(fft_result[:n // 2])
    frequencies = frequencies[:n // 2]
    
    # Compute the mean frequency (weighted by magnitude)
    mean_frequency = np.sum(frequencies * magnitude) / np.sum(magnitude)
    if plot == True:
        # Plot the frequency spectrum
        plt.figure(figsize=(10, 6))
        plt.plot(frequencies, magnitude, label='Frequency Spectrum')
        plt.axvline(mean_frequency, color='red', linestyle='--', label=f'Mean Frequency: {mean_frequency:.3f} Hz')
        plt.title('Frequency Spectrum')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Amplitude')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()
    return mean_frequency

def calculate_autocorrelation(data):
    """
    Calculate the autocorrelation function of a time series using FFT.
    """
    n = len(data)
    data -= np.mean(data)  # Remove mean to center the data
    fft_data = np.fft.fft(data, n=2*n)  # Compute FFT with zero-padding
    power_spectrum = np.abs(fft_data) ** 2  # Compute power spectrum
    autocorr = np.fft.ifft(power_spectrum).real[:n]  # Inverse FFT for autocorrelation
    autocorr /= autocorr[0]  # Normalize
    return autocorr

def find_lag(data):
    """
    Find the lag J where the autocorrelation function drops to 1 - 1/e of its initial value.
    """
    autocorr = calculate_autocorrelation(data)
    threshold = 1 - 1/np.e
    try:
        # Find the first lag where autocorrelation drops below the threshold
        j = np.where(autocorr <= threshold)[0][0]
    except IndexError:
        # If it never drops below, return None
        j = None
    return j

Mean Period is used as the min_tsep Parameter in nolds.lyap_r

In [8]:
min_tsep_mackey = round(1/determine_mean_frequency(data_mackey))
lag_mackey = find_lag(data_mackey)
min_tsep_lorenz_x = round(1/determine_mean_frequency(data_lorenz[:,0]))
lag_lorenz_x = find_lag(data_lorenz[:,0])
min_tsep_lorenz_y = round(1/determine_mean_frequency(data_lorenz[:,1]))
lag_lorenz_y = find_lag(data_lorenz[:,1])
min_tsep_lorenz_z = round(1/determine_mean_frequency(data_lorenz[:,2]))
lag_lorenz_z = find_lag(data_lorenz[:,2])

min_tsep_henon_x = round(1/determine_mean_frequency(data_henon[:,0]))
lag_henon_x = find_lag(data_henon[:,0])
min_tsep_henon_y = round(1/determine_mean_frequency(data_henon[:,1]))
lag_henon_y = find_lag(data_henon[:,1])
print("Mean Period of Mackey Glass: ", min_tsep_mackey)
print("Lag of Mackey Glass: ", lag_mackey)
print("Mean Period of Lorenz Attractor-x: ", min_tsep_lorenz_x)
print("Lag of Lorenz Attractor-x: ", lag_lorenz_x)
print("Mean Period of Lorenz Attractor-y: ", min_tsep_lorenz_y)
print("Lag of Lorenz Attractor-y: ", lag_lorenz_y)
print("Mean Period of Lorenz Attractor-z: ", min_tsep_lorenz_z)
print("Lag of Lorenz Attractor-z: ", lag_lorenz_z)
print("Mean Period of Henon Map-x: ", min_tsep_henon_x)
print("Lag of Henon Map-x: ", lag_henon_x)
print("Mean Period of Henon Map-y: ", min_tsep_henon_y)
print("Lag of Henon Map-y: ", lag_henon_y)

Mean Period of Mackey Glass:  44
Lag of Mackey Glass:  7
Mean Period of Lorenz Attractor-x:  19
Lag of Lorenz Attractor-x:  6
Mean Period of Lorenz Attractor-y:  17
Lag of Lorenz Attractor-y:  5
Mean Period of Lorenz Attractor-z:  22
Lag of Lorenz Attractor-z:  4
Mean Period of Henon Map-x:  4
Lag of Henon Map-x:  1
Mean Period of Henon Map-y:  4
Lag of Henon Map-y:  1


Mackey Glass Analysis: Change lag to 8 becasue for 7 the results are unstable

In [9]:
lyap_time_mackey = np.mean([1/nolds.lyap_r(data_mackey, emb_dim=12, min_tsep=min_tsep_mackey, lag=lag_mackey) for _ in range(100)])
print("Lyapunov Time of Mackey Glass: ", lyap_time_mackey)

Lyapunov Time of Mackey Glass:  139.95014987113433


Analyze Lorenz

In [10]:
lyap_time_lorenz_x = np.mean([1/nolds.lyap_r(data_lorenz[:,0], emb_dim=5, min_tsep=min_tsep_lorenz_x, lag=lag_lorenz_x) for _ in range(100)])
lyap_time_lorenz_y = np.mean([1/nolds.lyap_r(data_lorenz[:,1], emb_dim=5, min_tsep=min_tsep_lorenz_y, lag=lag_lorenz_y) for _ in range(100)])
lyap_time_lorenz_z = np.mean([1/nolds.lyap_r(data_lorenz[:,2], emb_dim=5, min_tsep=min_tsep_lorenz_z, lag=lag_lorenz_z) for _ in range(100)])
print("Lyapunov Time of Lorenz Attractor: ", np.mean([lyap_time_lorenz_x, lyap_time_lorenz_y, lyap_time_lorenz_z]))

Lyapunov Time of Lorenz Attractor:  24.85748432365534


Analyze Henon

In [11]:
lyap_time_henon_x = np.mean([1/nolds.lyap_r(data_henon[:,0], emb_dim=2, min_tsep=min_tsep_henon_x, lag=lag_henon_x) for _ in range(100)])
lyap_time_henon_y = np.mean([1/nolds.lyap_r(data_henon[:,1], emb_dim=2, min_tsep=min_tsep_henon_y, lag=lag_henon_y) for _ in range(100)])
print("Lyapunov Time of Henon Map: ", np.mean([lyap_time_henon_x, lyap_time_henon_y]))

Lyapunov Time of Henon Map:  3.3745284854054685
